# 데이터 입력 함수

In [1]:
# 프론트로부터 입력받는 데이터 (JSON)

import json
from datetime import datetime
from pathlib import Path

# 파일 경로 설정
DATA_DIR = Path.cwd() / "mock_data"
SONG_SESSIONS_FILE = DATA_DIR / "song_sessions.json"
PERSONS_AVAILABILITY_FILE = DATA_DIR / "persons_availability.json"
BASE_SCHEDULE_FILE = DATA_DIR / "base_schedule.json"
SESSION_WEIGHT_FILE = DATA_DIR / "session_weight.json"

# 곡별 세션 정보 불러오기
def load_song_sessions() -> dict:
    with open(SONG_SESSIONS_FILE, "r", encoding="utf-8") as f:
        return json.load(f)

# 사람별 가능한 시간 불러오기 (문자열 → datetime 변환)
def load_persons_availability() -> dict:
    with open(PERSONS_AVAILABILITY_FILE, "r", encoding="utf-8") as f:
        raw = json.load(f)
        return {
            name: [datetime.strptime(t, "%Y-%m-%d %H:%M") for t in times]
            for name, times in raw.items()
        }

# 기준 스케줄 불러오기 (문자열 → datetime 변환)
def load_base_schedule() -> list[datetime]:
    with open(BASE_SCHEDULE_FILE, "r", encoding="utf-8") as f:
        times = json.load(f)
        return [datetime.strptime(t, "%Y-%m-%d %H:%M") for t in times]

# 세션별 가중치 불러오기
def load_session_weight() -> dict:
    with open(SESSION_WEIGHT_FILE, "r", encoding="utf-8") as f:
        return json.load(f)


song_sessions = load_song_sessions()
persons_availability = load_persons_availability()
base_schedule = load_base_schedule()
session_weight = load_session_weight()

# 방 개수를 전역변수로 저장
rooms = 3


FileNotFoundError: [Errno 2] No such file or directory: 'c:\\Users\\ozxwx\\지원\\project\\mock_data\\song_sessions.json'

# 타임테이블 - 인원수 기준 & 가중치


In [ ]:
# sons_sessions(곡별 세션 정보)를 person_role(이름별 역할)로 정리한 딕셔너리 반환
def make_person_role(song_sessions: dict) -> dict:
    
    person_role = {}
    for song, sessions in song_sessions.items():
        for session, name in sessions.items():
            if name is None:
                continue
            if name not in person_role:
                person_role[name] = []
            person_role[name].append((song, session))
    return person_role

person_role = make_person_role(song_sessions)
print(person_role)

{'오지원': [('Violet', '보컬'), ('Violet', '기타2'), ('불', '보컬')], '윤진경': [('Violet', '기타1'), ('Holiday', '기타2'), ('Antifreeze', '기타2')], '신정현': [('Violet', '베이스'), ('ride', '베이스'), ('Antifreeze', '베이스'), ('불', '베이스')], '박수빈': [('Violet', '드럼'), ('ride', '드럼')], '곽민욱': [('ride', '보컬'), ('ride', '기타2'), ('찬란', '기타1'), ('불', '기타1')], '손호유': [('ride', '기타1'), ('Antifreeze', '기타1'), ('찬란', '기타2')], '차희태': [('ride', '건반'), ('불', '건반')], '이상혁': [('Holiday', '보컬')], '이재욱': [('Holiday', '기타1')], '송채우': [('Holiday', '베이스'), ('찬란', '베이스')], '장희찬': [('Holiday', '드럼'), ('찬란', '드럼')], '김강현': [('Antifreeze', '보컬'), ('찬란', '보컬')], '김정태': [('Antifreeze', '건반'), ('찬란', '건반')], '함채경': [('Antifreeze', '드럼'), ('불', '드럼')]}


In [ ]:
# class Person 만들기
# 속성: name, role, available_time

class Person:
    def __init__(self, name, role, available_time):
        self.name = name
        self.role = role
        self.available_time = available_time

    def __repr__(self):
        return f"Person(name={self.name}, role={self.role}, available_time={self.available_time})"


def make_people(person_role: dict, persons_availability: dict) -> list:
    people = []
    
    for name, roles in person_role.items():
        available_time = persons_availability.get(name, [])  # 없으면 빈 리스트
        person = Person(name, roles, available_time)
        people.append(person)
        
    return people
  
people = make_people(person_role,persons_availability)
print(people)

[Person(name=오지원, role=[('Violet', '보컬'), ('Violet', '기타2'), ('불', '보컬')], available_time=[datetime.datetime(2025, 5, 13, 17, 30), datetime.datetime(2025, 5, 13, 18, 0), datetime.datetime(2025, 5, 13, 18, 30), datetime.datetime(2025, 5, 13, 19, 0), datetime.datetime(2025, 5, 13, 19, 30)]), Person(name=윤진경, role=[('Violet', '기타1'), ('Holiday', '기타2'), ('Antifreeze', '기타2')], available_time=[datetime.datetime(2025, 5, 13, 18, 0), datetime.datetime(2025, 5, 13, 18, 30), datetime.datetime(2025, 5, 13, 19, 0), datetime.datetime(2025, 5, 13, 19, 30), datetime.datetime(2025, 5, 15, 18, 0), datetime.datetime(2025, 5, 15, 18, 30), datetime.datetime(2025, 5, 15, 19, 0), datetime.datetime(2025, 5, 15, 19, 30)]), Person(name=신정현, role=[('Violet', '베이스'), ('ride', '베이스'), ('Antifreeze', '베이스'), ('불', '베이스')], available_time=[datetime.datetime(2025, 5, 13, 18, 0), datetime.datetime(2025, 5, 13, 18, 30), datetime.datetime(2025, 5, 13, 19, 0), datetime.datetime(2025, 5, 13, 19, 30), datetime.datetime(

In [ ]:
# 공통되는 시간w/가중치 찾기
from collections import defaultdict
def sort_time_list(base_schedule: list, people: list, session_weight: dict) -> list:

    time_list = []

    for time in base_schedule:
        song_info = defaultdict(lambda: {'인원 수': 0, '가중치 합': 0})

        for person in people:
            if time in person.available_time:
                for song, session in person.role:
                    song_info[song]['인원 수'] += 1
                    song_info[song]['가중치 합'] += session_weight.get(session, 0)

        # song_info가 비어 있지 않으면 리스트에 튜플로 추가
        for song, info in song_info.items():
            time_list.append((time, song, info['인원 수'], info['가중치 합']))
    
    # result -> (시간, 곡, 인원수, 가중치) 튜플을 저장하는 리스트
    return time_list

time_list = sort_time_list(base_schedule, people, session_weight)
print(time_list)

[(datetime.datetime(2025, 5, 13, 17, 0), 'Holiday', 1, 8), (datetime.datetime(2025, 5, 13, 18, 30), 'Violet', 4, 16), (datetime.datetime(2025, 5, 13, 18, 30), '불', 2, 6), (datetime.datetime(2025, 5, 13, 18, 30), 'Holiday', 5, 32), (datetime.datetime(2025, 5, 13, 18, 30), 'Antifreeze', 3, 12), (datetime.datetime(2025, 5, 13, 18, 30), 'ride', 2, 10), (datetime.datetime(2025, 5, 13, 18, 30), '찬란', 3, 20), (datetime.datetime(2025, 5, 13, 18, 0), 'Violet', 4, 16), (datetime.datetime(2025, 5, 13, 18, 0), '불', 2, 6), (datetime.datetime(2025, 5, 13, 18, 0), 'Holiday', 4, 30), (datetime.datetime(2025, 5, 13, 18, 0), 'Antifreeze', 3, 12), (datetime.datetime(2025, 5, 13, 18, 0), 'ride', 2, 10), (datetime.datetime(2025, 5, 13, 18, 0), '찬란', 2, 18), (datetime.datetime(2025, 5, 13, 18, 30), 'Violet', 4, 16), (datetime.datetime(2025, 5, 13, 18, 30), '불', 2, 6), (datetime.datetime(2025, 5, 13, 18, 30), 'Holiday', 5, 32), (datetime.datetime(2025, 5, 13, 18, 30), 'Antifreeze', 3, 12), (datetime.datetime

In [ ]:
# 자동 스케줄링 with 인원 겹침 제거
def assign_schedule(time_list: list, person_role: dict, rooms: int, sort_by="count_first") -> dict:
    schedule = defaultdict(list)  # time -> list of (song, count, weight)
    assigned = set()  # (person, time) 중복 방지용
    
    # 정렬 기준 파라미터 (여러 기준 중 골라서 적용 가능)
    if sort_by == "count_first":
        sort_key = lambda x: (x[0], -x[2], -x[3])  # 시간, 인원수, 가중치 (기본)
    elif sort_by == "weight_first":
        sort_key = lambda x: (x[0], -x[3], -x[2])  # 시간, 가중치, 인원수
    else:
        sort_key = lambda x: (x[0], x[1])  # 시간, 곡 이름순 (방어용: 다른 단어 넣으면 적용됨)

    for time, song, count, weight in sorted(time_list, key=sort_key):
        if len(schedule[time]) >= rooms:
            continue

        conflict = False
        for person, roles in person_role.items():
            for role_song, _ in roles:
                if role_song == song:
                    if (person, time) in assigned:
                        conflict = True
                        break

        if not conflict:
            schedule[time].append((song, count, weight))
            for person, roles in person_role.items():
                for role_song, _ in roles:
                    if role_song == song:
                        assigned.add((person, time))

    return dict(schedule)

schedule = assign_schedule(time_list, person_role, rooms)
print(schedule)

{datetime.datetime(2025, 5, 13, 17, 0): [('Holiday', 1, 8)], datetime.datetime(2025, 5, 13, 18, 0): [('Holiday', 4, 30), ('Violet', 4, 16), ('Antifreeze', 3, 12)], datetime.datetime(2025, 5, 13, 18, 30): [('Holiday', 5, 32), ('Holiday', 5, 32), ('Violet', 4, 16)], datetime.datetime(2025, 5, 13, 19, 0): [('Holiday', 4, 24), ('Violet', 4, 16), ('찬란', 3, 20)], datetime.datetime(2025, 5, 13, 19, 30): [('Violet', 4, 16), ('Antifreeze', 3, 12), ('Holiday', 3, 8)], datetime.datetime(2025, 5, 13, 20, 0): [('Holiday', 2, 6), ('Antifreeze', 2, 3), ('찬란', 2, 3)], datetime.datetime(2025, 5, 13, 20, 30): [('Violet', 1, 2), ('ride', 1, 2), ('Antifreeze', 1, 2)], datetime.datetime(2025, 5, 15, 17, 0): [('Violet', 1, 16), ('ride', 1, 16)], datetime.datetime(2025, 5, 15, 17, 30): [('Violet', 1, 16), ('ride', 1, 16)], datetime.datetime(2025, 5, 15, 18, 0): [('Antifreeze', 3, 22), ('ride', 3, 22), ('Violet', 2, 24)], datetime.datetime(2025, 5, 15, 18, 30): [('Antifreeze', 3, 22), ('ride', 3, 22), ('Viole

In [ ]:
# 어느 시간에 어떤 곡에 누가 참여하고 불참하는지 return하는 함수
def assign_schedule_participant(
    schedule: dict,
    person_role: dict,
    persons_availability: dict
) -> dict:
    schedule_participant = {}

    for time in sorted(schedule):
        schedule_participant[time] = []

        for song, _, _ in schedule[time]:
            participants = []
            absentees = []

            for person, roles in person_role.items():
                for s, session in roles:
                    if s == song:
                        entry = f"{person}({session})"
                        if time in persons_availability.get(person, []):
                            participants.append(entry)
                        else:
                            absentees.append(entry)

            schedule_participant[time].append({
                song: [participants, absentees] # 리스트[0] -> 참여자, 리스트[1] -> 불참자
            })

    return schedule_participant

schedule_participant = assign_schedule_participant(schedule, person_role, persons_availability)
print(schedule_participant)

{datetime.datetime(2025, 5, 13, 17, 0): [{'Holiday': [['이재욱(기타1)'], ['윤진경(기타2)', '이상혁(보컬)', '송채우(베이스)', '장희찬(드럼)']]}], datetime.datetime(2025, 5, 13, 18, 0): [{'Holiday': [['윤진경(기타2)', '이상혁(보컬)', '이재욱(기타1)', '장희찬(드럼)'], ['송채우(베이스)']]}, {'Violet': [['오지원(보컬)', '오지원(기타2)', '윤진경(기타1)', '신정현(베이스)'], ['박수빈(드럼)']]}, {'Antifreeze': [['윤진경(기타2)', '신정현(베이스)', '손호유(기타1)'], ['김강현(보컬)', '김정태(건반)', '함채경(드럼)']]}], datetime.datetime(2025, 5, 13, 18, 30): [{'Holiday': [['윤진경(기타2)', '이상혁(보컬)', '이재욱(기타1)', '송채우(베이스)', '장희찬(드럼)'], []]}, {'Holiday': [['윤진경(기타2)', '이상혁(보컬)', '이재욱(기타1)', '송채우(베이스)', '장희찬(드럼)'], []]}, {'Violet': [['오지원(보컬)', '오지원(기타2)', '윤진경(기타1)', '신정현(베이스)'], ['박수빈(드럼)']]}], datetime.datetime(2025, 5, 13, 19, 0): [{'Holiday': [['윤진경(기타2)', '이상혁(보컬)', '송채우(베이스)', '장희찬(드럼)'], ['이재욱(기타1)']]}, {'Violet': [['오지원(보컬)', '오지원(기타2)', '윤진경(기타1)', '신정현(베이스)'], ['박수빈(드럼)']]}, {'찬란': [['손호유(기타2)', '송채우(베이스)', '장희찬(드럼)'], ['곽민욱(기타1)', '김강현(보컬)', '김정태(건반)']]}], datetime.datetime(2025, 5, 13, 19, 30): [{'

In [ ]:
# schedule_participant에서, 각 시간당 불참자가 최소인 곡을 배정
# 프론트로 보내서 바로 시각화할 수 있도록 결과 포맷 지정
# 프론트에서는 각 시간마다 곡이 배정된 타임테이블 형식으로 시각화하고,
# 클릭 등을 통해 각 시간마다 참여자/불참자를 확인할 수 있음음
def summarize_for_timetable(schedule_participant: dict) -> dict:
    timetable = {}

    for time, entries in schedule_participant.items():
        best_song = None
        min_absent = float("inf")
        best_pair = [[], []]  # [participants, absentees]

        for entry in entries:
            for song, (participants, absentees) in entry.items():
                if len(absentees) < min_absent:
                    min_absent = len(absentees)
                    best_song = song
                    best_pair = [participants, absentees]

        timetable[time] = (best_song, best_pair)

    return timetable
timetable = summarize_for_timetable(schedule_participant)
print(timetable)

{datetime.datetime(2025, 5, 13, 17, 0): ('Holiday', [['이재욱(기타1)'], ['윤진경(기타2)', '이상혁(보컬)', '송채우(베이스)', '장희찬(드럼)']]), datetime.datetime(2025, 5, 13, 18, 0): ('Holiday', [['윤진경(기타2)', '이상혁(보컬)', '이재욱(기타1)', '장희찬(드럼)'], ['송채우(베이스)']]), datetime.datetime(2025, 5, 13, 18, 30): ('Holiday', [['윤진경(기타2)', '이상혁(보컬)', '이재욱(기타1)', '송채우(베이스)', '장희찬(드럼)'], []]), datetime.datetime(2025, 5, 13, 19, 0): ('Holiday', [['윤진경(기타2)', '이상혁(보컬)', '송채우(베이스)', '장희찬(드럼)'], ['이재욱(기타1)']]), datetime.datetime(2025, 5, 13, 19, 30): ('Violet', [['오지원(보컬)', '오지원(기타2)', '윤진경(기타1)', '신정현(베이스)'], ['박수빈(드럼)']]), datetime.datetime(2025, 5, 13, 20, 0): ('Holiday', [['이상혁(보컬)', '송채우(베이스)'], ['윤진경(기타2)', '이재욱(기타1)', '장희찬(드럼)']]), datetime.datetime(2025, 5, 13, 20, 30): ('Violet', [['신정현(베이스)'], ['오지원(보컬)', '오지원(기타2)', '윤진경(기타1)', '박수빈(드럼)']]), datetime.datetime(2025, 5, 15, 17, 0): ('Violet', [['박수빈(드럼)'], ['오지원(보컬)', '오지원(기타2)', '윤진경(기타1)', '신정현(베이스)']]), datetime.datetime(2025, 5, 15, 17, 30): ('Violet', [['박수빈(드럼)'], ['오지

# 시각화 함수 (터미널 & 콘솔용)


In [ ]:
# 시각화 함수
# 디버깅할 때 데이터 형태 확인용
# 변수에는 schedule, sorted_schedule 사용 가능
def visualize_schedule(schedule: dict) -> str:
    result = ""
    for time in sorted(schedule):
        result += f"[시간: {time}]\n"
        for song, count, weight in schedule[time]:
            result += f"  - {song}\t| 인원 수: {count}\t| 가중치: {weight}\n"
        result += "\n"
    return result

print(visualize_schedule(schedule))

[시간: 2025-05-13 17:00:00]
  - Holiday	| 인원 수: 1	| 가중치: 8

[시간: 2025-05-13 18:00:00]
  - Holiday	| 인원 수: 4	| 가중치: 30
  - Violet	| 인원 수: 4	| 가중치: 16
  - Antifreeze	| 인원 수: 3	| 가중치: 12

[시간: 2025-05-13 18:30:00]
  - Holiday	| 인원 수: 5	| 가중치: 32
  - Holiday	| 인원 수: 5	| 가중치: 32
  - Violet	| 인원 수: 4	| 가중치: 16

[시간: 2025-05-13 19:00:00]
  - Holiday	| 인원 수: 4	| 가중치: 24
  - Violet	| 인원 수: 4	| 가중치: 16
  - 찬란	| 인원 수: 3	| 가중치: 20

[시간: 2025-05-13 19:30:00]
  - Violet	| 인원 수: 4	| 가중치: 16
  - Antifreeze	| 인원 수: 3	| 가중치: 12
  - Holiday	| 인원 수: 3	| 가중치: 8

[시간: 2025-05-13 20:00:00]
  - Holiday	| 인원 수: 2	| 가중치: 6
  - Antifreeze	| 인원 수: 2	| 가중치: 3
  - 찬란	| 인원 수: 2	| 가중치: 3

[시간: 2025-05-13 20:30:00]
  - Violet	| 인원 수: 1	| 가중치: 2
  - ride	| 인원 수: 1	| 가중치: 2
  - Antifreeze	| 인원 수: 1	| 가중치: 2

[시간: 2025-05-15 17:00:00]
  - Violet	| 인원 수: 1	| 가중치: 16
  - ride	| 인원 수: 1	| 가중치: 16

[시간: 2025-05-15 17:30:00]
  - Violet	| 인원 수: 1	| 가중치: 16
  - ride	| 인원 수: 1	| 가중치: 16

[시간: 2025-05-15 18:00:00]
  - Antifreeze	| 

In [ ]:
# schedule에서 시간마다 참여 가능한 사람들을 시각화하는 함수
def visualize_participation(schedule_participant: dict) -> str:
    result = ""

    for time in sorted(schedule_participant):
        result += f"[시간: {time}]\n"
        for song_entry in schedule_participant[time]:
            for song, (participants, absentees) in song_entry.items():
                participants = ", ".join(participants) if participants else "없음"
                absentees = ", ".join(absentees) if absentees else "없음"

                result += f"- {song}\n"
                result += f"  - 참여: {participants}\n"
                result += f"  - 불참: {absentees}\n\n"

    return result


print(visualize_participation(schedule_participant))

[시간: 2025-05-13 17:00:00]
- Holiday
  - 참여: 이재욱(기타1)
  - 불참: 윤진경(기타2), 이상혁(보컬), 송채우(베이스), 장희찬(드럼)

[시간: 2025-05-13 18:00:00]
- Holiday
  - 참여: 윤진경(기타2), 이상혁(보컬), 이재욱(기타1), 장희찬(드럼)
  - 불참: 송채우(베이스)

- Violet
  - 참여: 오지원(보컬), 오지원(기타2), 윤진경(기타1), 신정현(베이스)
  - 불참: 박수빈(드럼)

- Antifreeze
  - 참여: 윤진경(기타2), 신정현(베이스), 손호유(기타1)
  - 불참: 김강현(보컬), 김정태(건반), 함채경(드럼)

[시간: 2025-05-13 18:30:00]
- Holiday
  - 참여: 윤진경(기타2), 이상혁(보컬), 이재욱(기타1), 송채우(베이스), 장희찬(드럼)
  - 불참: 없음

- Holiday
  - 참여: 윤진경(기타2), 이상혁(보컬), 이재욱(기타1), 송채우(베이스), 장희찬(드럼)
  - 불참: 없음

- Violet
  - 참여: 오지원(보컬), 오지원(기타2), 윤진경(기타1), 신정현(베이스)
  - 불참: 박수빈(드럼)

[시간: 2025-05-13 19:00:00]
- Holiday
  - 참여: 윤진경(기타2), 이상혁(보컬), 송채우(베이스), 장희찬(드럼)
  - 불참: 이재욱(기타1)

- Violet
  - 참여: 오지원(보컬), 오지원(기타2), 윤진경(기타1), 신정현(베이스)
  - 불참: 박수빈(드럼)

- 찬란
  - 참여: 손호유(기타2), 송채우(베이스), 장희찬(드럼)
  - 불참: 곽민욱(기타1), 김강현(보컬), 김정태(건반)

[시간: 2025-05-13 19:30:00]
- Violet
  - 참여: 오지원(보컬), 오지원(기타2), 윤진경(기타1), 신정현(베이스)
  - 불참: 박수빈(드럼)

- Antifreeze
  - 참여: 윤진경(기타2), 신정현(베이스), 손호유(기타